# Back to the patch clamp!

In [1]:
import patches
import lettertask as lt
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import itertools
import pandas as pd

## The hidden markov model

In [2]:
data = lt.data.CompositionalBinaryModel(samples=1000)

In [3]:
input_data = data.to_array().reshape(1, *data.to_array().shape)

In [4]:
latent_data = data.latent_array().reshape(1, *data.latent_array().shape)

In [5]:
dataset = patches.data.HiddenMarkovModel(input_data, latent_data)

In [6]:
dataset[0]['input'].shape

torch.Size([1, 100])

Looks good!

## General structure

In [7]:
def model_generator(input_features, latent_features, timesteps, intermediate_features, **kwargs):
    encoder = nn.Sequential(
        nn.Linear(input_features, intermediate_features),
        nn.ReLU(),
        nn.Linear(intermediate_features, latent_features)
    )
    predictor = nn.Linear(latent_features, timesteps*latent_features)
    decoder = nn.Linear(latent_features, input_features)
    return {
        'encoder': encoder,
        'predictor': predictor,
        'decoder': decoder
    }

In [8]:
model_hyperparameters = {
    'timesteps': [1, 5, 10],
    'intermediate_features': [1, 2, 3, 4, 5, 10]
}

In [9]:
patchclamp = patches.patchclamp.PatchClamp(model_generator, model_hyperparameters)

In [10]:
scr = patchclamp.get_scr(100, 1, timesteps=5, intermediate_features=2)

In [11]:
scr(dataset[0])

tensor([[0.2706]], grad_fn=<AddmmBackward>)

In [12]:
scr.loss(scr(dataset[0]), dataset[0], nn.MSELoss())

tensor(1.6143, grad_fn=<MseLossBackward>)

In [13]:
cc = patchclamp.get_cc(100, 1, timesteps=5, intermediate_features=2)

In [14]:
contrastive_dataset = patches.data.ContrastiveDataset(input_data, contrast_type='both',
                                                      prediction_range=5)

In [15]:
cc.loss(cc(contrastive_dataset[0]), contrastive_dataset[0], nn.MSELoss())

tensor(0.5718, grad_fn=<MeanBackward0>)

In [44]:
params = list(cc.parameters())[0].detach().numpy()

In [49]:
params = params/np.sqrt((params**2).sum(axis=1)).reshape(params.shape[0], 1)

In [55]:
ideals = np.diag(1/np.sqrt(np.array([100]))).repeat(np.array([100]), axis=1)

In [61]:
help(np.save)

Help on function save in module numpy:

save(file, arr, allow_pickle=True, fix_imports=True)
    Save an array to a binary file in NumPy ``.npy`` format.
    
    Parameters
    ----------
    file : file, str, or pathlib.Path
        File or filename to which the data is saved.  If file is a file-object,
        then the filename is unchanged.  If file is a string or Path, a ``.npy``
        extension will be appended to the file name if it does not already
        have one.
    arr : array_like
        Array data to be saved.
    allow_pickle : bool, optional
        Allow saving object arrays using Python pickles. Reasons for disallowing
        pickles include security (loading pickled data can execute arbitrary
        code) and portability (pickled objects may not be loadable on different
        Python installations, for example if the stored objects require libraries
        that are not available, and not all pickled data is compatible between
        Python 2 and Python 3).
 

## Preparations for experiment 20

## Studied models

In [16]:
widths = [
    [5, 5],
    [10, 10],
    [50, 50],
    [5, 10],
    [5, 50],
    [10, 50],
    [100, 100],
    [1000, 1000],
    [5, 5, 5],
    [10, 10, 10],
    [50, 50, 50],
    [5, 10, 10],
    [5, 10, 50],
    [50, 50, 50],
    [100, 100, 100],
    [100, 200, 200],
    [5, 5, 5, 5, 5],
    [5, 50, 50, 50, 50], 
    [100, 50, 100, 50, 100]
]

In [17]:
change_probabilities = [
    [0.05, 0.5],
    [0.05, 0.2],
    [0.05, 0.1],
    [0.2, 0.5],
    [0.05, 0.1, 0.5],
    [0.05, 0.2, 0.5],
    [0.05, 0.08, 0.1],
    [0.05, 0.1, 0.2, 0.4, 0.5],
    [0.05, 0.08, 0.1, 0.2, 0.3],
    [0.05, 0.4, 0.45, 0.48, 0.5]
]

In [18]:
model_combinations = [[width, change_probability]\
                      for width in widths\
                      for change_probability in change_probabilities\
                      if len(width) == len(change_probability)]

In [19]:
rs = np.random.RandomState(seed=10000)

In [20]:
seeds = rs.choice(range(10000), size=100)

In [22]:
samples = [
    100, 
    1000,
    1e4,
    1e5,
    5e5
]

## Optimizers

For larger models, we should implement a random search rather than a grid search, but for such a small model, it seems nice to be apply to easily marginalize different dimensions.

For now, however, we only use learning rate as a hyperparameter.

In [23]:
learning_rates = [
    1e-6,
    1e-5,
    1e-4, 2e-4, 5e-4,
    1e-3, 2e-3, 5e-3,
    1e-2, 2e-2, 5e-2,
    1e-1, 5e-1, 1
]

In [24]:
methods = ['SGD', 'Adam', 'Adadelta', 'Adamax']

## Putting the data frame together

In [32]:
width = [5, 10]

In [40]:
ideals = np.diag(1/np.sqrt(np.array(width))).repeat(np.array(width), axis=1)

In [62]:
method = optim.SGD

In [64]:
help(lt.data.CompositionalBinaryModel)

Help on class CompositionalBinaryModel in module lettertask.data.atomic_model:

class CompositionalBinaryModel(builtins.object)
 |  CompositionalBinaryModel(width=[100], change_probability=[0.05], samples=0, seed=None, random_state=None)
 |  
 |  The compositional binary model.
 |  
 |  This class models the compositional binary model, which consists of several
 |  atomic binary models.
 |  
 |  Args:
 |      width: How many columns should the matrix at each timestep have? (
 |          Default is 100.)
 |      change_probability: What should be the probability of the number
 |          changing in the first row? (Default is 0.05.)
 |      presamples: How many samples should be drawn initially? Further samples
 |          can always be drawn using the method 'sample'. (Default is 0.)
 |      seed: For reproducibility, you can provide a random seed.
 |  
 |  Raises:
 |      ValueError: If width is not a positive integer, change_probability is
 |          not a valid probability, presamp